In [ ]:
from pathlib import Path
import json

pred_path = Path("infoseek_memtree_predictions_all_5k_collapsed_beta_0.6_top10.jsonl")
records = [json.loads(line) for line in pred_path.open("r", encoding="utf-8")]
print(f"Loaded {len(records)} predictions from {pred_path}")

In [6]:
from collections import defaultdict

def compute_metrics(examples):
    total = len(examples)
    tp = sum(1 for ex in examples if ex.get("is_correct"))
    fp = 0
    fn = total - tp
    accuracy = tp / total if total else 0.0
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    recall = tp / (tp + fn) if (tp + fn) else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
    return {"total": total, "correct": tp, "accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

overall_metrics = compute_metrics(records)
print("Overall:", overall_metrics)

split_buckets = defaultdict(list)
for rec in records:
    split_buckets[rec["data_split"]].append(rec)

for split, examples in split_buckets.items():
    print(f"{split}: {compute_metrics(examples)}")

Overall: {'total': 5000, 'correct': 1934, 'accuracy': 0.3868, 'precision': 1.0, 'recall': 0.3868, 'f1': 0.557830977790597}
val_unseen_question: {'total': 1309, 'correct': 504, 'accuracy': 0.3850267379679144, 'precision': 1.0, 'recall': 0.3850267379679144, 'f1': 0.555984555984556}
val_unseen_entity: {'total': 3691, 'correct': 1430, 'accuracy': 0.3874288810620428, 'precision': 1.0, 'recall': 0.3874288810620428, 'f1': 0.5584846709627026}


In [ ]:
import re
from collections import defaultdict

def normalize_answer(text):
    if text is None:
        return ""
    if not isinstance(text, str):
        text = str(text)
    text = text.strip().lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    return re.sub(r"\s+", " ", text)

def quick_match(prediction, gold_answers):
    if not prediction:
        return False
    pred_norm = normalize_answer(prediction)
    if not pred_norm:
        return False
    for ans in gold_answers:
        if not ans:
            continue
        gold_norm = normalize_answer(ans)
        if gold_norm and (gold_norm in pred_norm or pred_norm in gold_norm):
            return True
    return False

def get_gold_answers(record):
    gold = record.get("answer_eval") or record.get("answer") or []
    if isinstance(gold, str):
        gold = [gold]
    return gold

for rec in records:
    rec["is_correct_substring"] = quick_match(rec["model_answer"], get_gold_answers(rec))

In [ ]:
# Cell 3: 计算整体与分类型指标
def compute_metrics(examples, key):
    total = len(examples)
    tp = sum(1 for ex in examples if ex[key])
    fp = 0
    fn = total - tp
    accuracy = tp / total if total else 0.0
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    recall = tp / (tp + fn) if (tp + fn) else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
    return {"total": total, "correct": tp, "accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

overall = compute_metrics(records, "is_correct_substring")
print("Overall:", overall)

split_buckets = defaultdict(list)
for rec in records:
    split_buckets[rec["data_split"]].append(rec)

for split, examples in split_buckets.items():
    print(split, compute_metrics(examples, "is_correct_substring"))